In [1]:
import pandas as pd
import numpy as np
import altair as alt
from os.path import join

from constants import COLUMNS
from utils import read_latest_labs_df, read_loinc_df

alt.data_transformers.disable_max_rows();

In [2]:
df = read_latest_labs_df()
df.head()

,siteid,loinc,days_since_positive,num_patients,mean_value,stdev_value
0,FZU,1751-7,-6,334,6.564976,6.869472
1,FZU,7799-0,-6,668,4.864037,3.641041
2,FZU,1988-5,-6,346,1.377003,-0.308969
3,FZU,33959-8,-6,523,7.741650,6.450967
4,FZU,1920-8,-6,862,11.535845,7.310155


In [3]:
loinc_df = read_loinc_df().set_index('loinc').rename(columns={'labTest': 'name'})
df["loinc_name"] = df[COLUMNS.LOINC].apply(lambda code: code + " (" + loinc_df.at[code, "name"] + ")")

In [4]:
line = alt.Chart(df).mark_line().encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)

point = alt.Chart(df).mark_point(filled=True).encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)


band = alt.Chart(df).mark_errorbar().encode(
    x=alt.X(COLUMNS.DAYS_SINCE_POSITIVE, title="Days since positive"),
    y=alt.Y(COLUMNS.MEAN_VALUE, title="Mean value"),
    yError=alt.YError(COLUMNS.STDEV_VALUE),
    color=alt.Color(COLUMNS.SITE_ID, title="Site")
)

(band + point + line).properties(
    width=150,
    height=150
).facet(
    facet=alt.Facet("loinc_name", title="LOINC"),
    columns=3
)

alt.FacetChart(...)